In [4]:
import pandas as pd
import os

spam_data = pd.read_csv("Spam-Classification.csv")
spam_data.head()

,CLASS,SMS
0,ham,"said kiss, kiss, i can't do the sound effects..."
1,ham,&lt;#&gt; ISH MINUTES WAS 5 MINUTES AGO. WTF.
2,spam,(Bank of Granite issues Strong-Buy) EXPLOSIVE ...
3,spam,* FREE* POLYPHONIC RINGTONE Text SUPER to 8713...
4,spam,**FREE MESSAGE**Thanks for using the Auction S...


In [5]:
# separate feature and target
spam_classes_raw = spam_data["CLASS"]
spam_messages = spam_data["SMS"]

In [6]:
# preprocessing data
from sklearn import preprocessing
import tensorflow as tf

label_encoder = preprocessing.LabelEncoder()
spam_classes = label_encoder.fit_transform(spam_classes_raw)

# convert target to one-hot encoding
spam_classes = tf.keras.utils.to_categorical(spam_classes,2)
print("One-hot encoding : ", spam_classes.shape)

One-hot encoding :  (1500, 2)


In [9]:
#Preprocess data for spam messages
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Max words in the vocabulary for this dataset
VOCAB_WORDS=10000
#Max sequence length for word sequences
MAX_SEQUENCE_LENGTH=100

#Create a vocabulary with unique words and IDs
spam_tokenizer = Tokenizer(num_words=VOCAB_WORDS)
spam_tokenizer.fit_on_texts(spam_messages)


print("Total unique tokens found: ", len(spam_tokenizer.word_index))
print("Example token ID for word \"me\" :", spam_tokenizer.word_index.get("me"))

#Convert sentences to token-ID sequences
spam_sequences = spam_tokenizer.texts_to_sequences(spam_messages)

#Pad all sequences to fixed length
spam_padded = pad_sequences(spam_sequences, maxlen=MAX_SEQUENCE_LENGTH)

print("\nTotal sequences found : ", len(spam_padded))
print("Example Sequence for sentence : ", spam_messages[0] )
print(spam_padded[0])

Total unique tokens found:  4688
Example token ID for word "me" : 25

Total sequences found :  1500
Example Sequence for sentence :   said kiss, kiss, i can't do the sound effects! He is a gorgeous man isn't he! Kind of person who needs a smile to brighten his day! 
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0  260  921  921    4  430   55    6 1488 2294  148   10
    3 1489  464 1143  148  922   19  514   77 1144    3  515    1 2295
  397   89]


In [10]:

#Split into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
                                    spam_padded,spam_classes,test_size=0.2)

In [11]:
#Load the pre-trained embeddings

import numpy as np

#Read pretrained embeddings into a dictionary
glove_dict = {} 

#Loading a 50 feature (dimension) embedding with 6 billion words
with open('glove.6B.50d.txt/glove.6B.50d.txt', "r", encoding="utf8") as glove_file:     
    for line in glove_file:
        
        emb_line = line.split()      
        emb_token = emb_line[0]         
        emb_vector = np.array(emb_line[1:], dtype=np.float32)
        
        if emb_vector.shape[0] == 50:    
            glove_dict[emb_token] = emb_vector 

print("Dictionary Size: ", len(glove_dict))
print("\n Sample Dictionary Entry for word \"the\" :\n", glove_dict.get("the"))

Dictionary Size:  400000

 Sample Dictionary Entry for word "the" :
 [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [12]:
#We now associate each token ID in our data set vocabulary to the corresponding embedding in Glove
#If the word is not available, then embedding will be all zeros.

#Matrix with 1 row for each word in the data set vocubulary and 50 features

vocab_len = len(spam_tokenizer.word_index) + 1

embedding_matrix = np.zeros((vocab_len, 50))

for word, id in spam_tokenizer.word_index.items():  
    try:
        embedding_vector = glove_dict.get(word) 
        if embedding_vector is not None:         
            embedding_matrix[id] = embedding_vector
    except:
        pass

print("Size of Embedding matrix :", embedding_matrix.shape)
print("Embedding Vector for word \"me\" : \n", embedding_matrix[spam_tokenizer.word_index.get("me")])

Size of Embedding matrix : (4689, 50)
Embedding Vector for word "me" : 
 [-0.14524999  0.31265     0.15184    -0.63708001  0.63552999 -0.50295001
 -0.23214     0.52891999 -0.58629     0.53934997 -0.3055      1.03569996
 -0.77989    -0.19386999  1.22150004  0.24521001  0.26144001  0.22439
  0.15583999 -0.79145998 -0.65262002  1.3211      0.76617998  0.38234001
  1.44529998 -2.26430011 -1.15050006  0.50373     1.2651     -1.59029996
  3.05180001  0.84118003 -0.69542998  0.29984999 -0.49151    -0.22312
  0.59527999 -0.076347    0.52358001 -0.50133997  0.22483     0.01546
 -0.088005    0.21281999  0.28545001 -0.15976    -0.16777    -0.50895
  0.14322001  1.01180005]


In [13]:
#Create a model
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
from keras.layers import LSTM,Dense

#Setup Hyper Parameters for building the model
NB_CLASSES=2

model = tf.keras.models.Sequential()

model.add(keras.layers.Embedding(vocab_len,
                                 50, 
                                 name="Embedding-Layer",
                                 weights=[embedding_matrix],
                                 input_length=MAX_SEQUENCE_LENGTH,
                                 trainable=True))

#Add LSTM Layer
model.add(LSTM(256))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding-Layer (Embedding  (None, 100, 50)           234450    
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 256)               314368    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 Output-Layer (Dense)        (None, 2)                 514       
                                                                 
Total params: 549332 (2.10 MB)
Trainable params: 549332 (2.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
#Make it verbose so we can see the progress
VERBOSE=1

#Setup Hyper Parameters for training
BATCH_SIZE=256
EPOCHS=10
VALIDATION_SPLIT=0.2

print("\nTraining Progress:\n------------------------------------")

history=model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

print("\nEvaluation against Test Dataset :\n------------------------------------")
model.evaluate(X_test,Y_test)


Training Progress:
------------------------------------
Epoch 1/10
4/4 [==============================] - 8s 1s/step - loss: 0.6159 - accuracy: 0.6010 - val_loss: 0.3804 - val_accuracy: 0.8750
Epoch 2/10
4/4 [==============================] - 5s 1s/step - loss: 0.2869 - accuracy: 0.9073 - val_loss: 0.5514 - val_accuracy: 0.7417
Epoch 3/10
4/4 [==============================] - 5s 1s/step - loss: 0.2875 - accuracy: 0.8969 - val_loss: 0.1947 - val_accuracy: 0.9250
Epoch 4/10
4/4 [==============================] - 5s 1s/step - loss: 0.1686 - accuracy: 0.9417 - val_loss: 0.1805 - val_accuracy: 0.9375
Epoch 5/10
4/4 [==============================] - 5s 1s/step - loss: 0.1993 - accuracy: 0.9354 - val_loss: 0.1603 - val_accuracy: 0.9458
Epoch 6/10
4/4 [==============================] - 4s 1s/step - loss: 0.1417 - accuracy: 0.9531 - val_loss: 0.2616 - val_accuracy: 0.9167
Epoch 7/10
4/4 [==============================] - 4s 1s/step - loss: 0.1627 - accuracy: 0.9458 - val_loss: 0.1543 - val_a

[0.15836729109287262, 0.9599999785423279]

In [15]:
# Two input strings to predict
input_str=["Unsubscribe send GET EURO STOP to 83222",
            "Yup I will come over"]

#Convert to sequence using the same tokenizer as training
input_seq = spam_tokenizer.texts_to_sequences(input_str)
#Pad the input
input_padded = pad_sequences(input_seq, maxlen=MAX_SEQUENCE_LENGTH)

#Predict using model
prediction=np.argmax( model.predict(input_padded), axis=1 )
print("Prediction Output:" , prediction)

#Print prediction classes
print("Prediction Classes are ", label_encoder.inverse_transform(prediction))

1/1 [==============================] - 1s 603ms/step
Prediction Output: [1 0]
Prediction Classes are  ['spam' 'ham']
